# Si/Si3N4 - Approximate

## import & prepare

In [1]:
!pip install git+https://github.com/nmdl-mizo/interface_master.git@develop
!mkdir -p cif_files
!curl https://raw.githubusercontent.com/nmdl-mizo/interface_master/develop/test_files/cif_files/Si_mp-149_conventional_standard.cif -o 'cif_files/Si_mp-149_conventional_standard.cif'
!curl https://raw.githubusercontent.com/nmdl-mizo/interface_master/develop/test_files/cif_files/Si3N4_mp-988_conventional_standard.cif -o 'cif_files/Si3N4_mp-988_conventional_standard.cif'

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/nmdl-mizo/interface_master.git (to revision develop) to /tmp/pip-req-build-oekxly57
  Running command git clone --filter=blob:none --quiet https://github.com/nmdl-mizo/interface_master.git /tmp/pip-req-build-oekxly57
  Resolved https://github.com/nmdl-mizo/interface_master.git to commit 2bdf96ed4835b036557ac18de82b4fa2afdabf24
  Preparing metadata (setup.py) ... done

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1059  100  1059    0     0  17650      0 --:--:-- --:--:-- --:--:-- 17650
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1371  100    

## Define an interface core class 

In [2]:
from interfacemaster.cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, get_normal_from_MI, rot
from interfacemaster.interface_generator import core, convert_vector_index, get_disorientation
from numpy import array, dot, round, cross, eye
from numpy.linalg import inv, det, norm

In [3]:
my_interface = core('cif_files/Si_mp-149_conventional_standard.cif',\
                    'cif_files/Si3N4_mp-988_conventional_standard.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


/Users/jasonxie/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1121: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))


In [4]:
#get the target disorientation
R = get_disorientation(L1 = my_interface.conv_lattice_1, L2 = my_interface.conv_lattice_2, \
                       v1 = [-1,1,0], hkl1 = [1, 1, 1], \
                       v2 = [1,0,0], hkl2 = [0, 0, 1])
R

array([[-7.07106781e-01, -4.08248290e-01,  5.77350269e-01],
       [ 7.07106781e-01, -4.08248290e-01,  5.77350269e-01],
       [-3.53525080e-17,  8.16496581e-01,  5.77350269e-01]])

In [8]:
#specify the two combining surfaces
hkl_1 = get_primitive_hkl([1, 1, 1], my_interface.conv_lattice_1, my_interface.lattice_1)
hkl_2 = get_primitive_hkl([0, 0, 1], my_interface.conv_lattice_2, my_interface.lattice_2)

## Searching two_dimensional CSL

In [9]:
my_interface.parse_limit(du = 1e-1, S  = 1e-1, sgm1=100, sgm2=100, dd = 1e-1)

#Do searching!
my_interface.search_one_position_2D(hkl_1, hkl_2, theta_range = 2, \
                                    dtheta = 0.01, pre_R = R, match_tol = 0.01, pre_dt= True)

Congrates, we found an appx CSL!

U1 = 
[[ 0 -2]
 [-2  2]
 [ 2  0]]; sigma_1 = 7

U2 = 
[[ 0  0]
 [ 1  0]
 [ 1 -1]]; sigma_2 = 1

D = 
[[ 1.00640892 -0.00320446 -0.00320446]
 [-0.00320446  1.00640892 -0.00320446]
 [-0.00320446 -0.00320446  1.00640892]]

axis = [-7.47674648 -7.47674648 -7.47674648] ; theta = 0.0



## Get bicrystal

In [10]:
my_interface.compute_bicrystal_two_D(hkl_1 = hkl_1, hkl_2=hkl_2, normal_ortho = True, plane_ortho = False, lim = 50, tol_ortho = 1e-2, tol_integer=1e-3)

cell 1:
[[-1  0 -2]
 [-1 -2  2]
 [-1  2  0]]
cell 2:
[[-1  0  0]
 [ 0  1  0]
 [ 0  1 -1]]


In [28]:
my_interface.get_bicrystal(two_D = True, xyz_1 = [1,1,1], xyz_2 = [1,1,1])

Please check the POSCAR file